# 0. Install

In [144]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash


Updating fastai...
Done.


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [0]:
from fastai import *
from fastai.text import *
import sklearn.feature_extraction.text as sklearn_text

# 2. Dataset

In [194]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/content/data/imdb_sample')

In [195]:
path.ls()

[PosixPath('/content/data/imdb_sample/texts.csv')]

In [196]:
df = pd.read_csv(path/'texts.csv')
df.shape

(1000, 3)

In [197]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


# 3. Preprocessing

In [0]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text').split_from_df(col=2).label_from_df(cols=0))

In [199]:
movie_reviews.valid.y[0], movie_reviews.valid.x[0]

(Category positive,
 Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
  
   xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
  
   xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj

In [200]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [201]:
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

(6008, 19161)

In [202]:
movie_reviews.vocab.stoi['love']

142

In [203]:
movie_reviews.vocab.itos[142]

'love'

In [204]:
movie_reviews.vocab.itos[140:149]

['life', 'characters', 'love', 'your', 'here', 'know', 'scenes', 'best', 'end']

In [208]:
movie_reviews.vocab.itos[-20:]

['uncritical',
 'whack',
 'heavies',
 'poe',
 'monstervision',
 'penn',
 'shrine',
 'lupino',
 'pegg',
 'grasshoppers',
 'flik',
 'ladder',
 'wtc',
 'portuguese',
 'della',
 'contractor',
 'coaxes',
 'mabuse',
 'greyson',
 'sollett']

In [209]:
movie_reviews.vocab.itos[:20]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i']

In [210]:
from itertools import *
list(islice(movie_reviews.vocab.stoi.items(), 20))

[('xxunk', 0),
 ('xxpad', 1),
 ('xxbos', 2),
 ('xxeos', 3),
 ('xxfld', 4),
 ('xxmaj', 5),
 ('xxup', 6),
 ('xxrep', 7),
 ('xxwrep', 8),
 ('the', 9),
 ('.', 10),
 (',', 11),
 ('and', 12),
 ('a', 13),
 ('of', 14),
 ('to', 15),
 ('is', 16),
 ('it', 17),
 ('in', 18),
 ('i', 19)]

Out of vocab Dictionary = Unknown

In [213]:
i = movie_reviews.vocab.stoi['bualabs']
movie_reviews.vocab.itos[i], i

('xxunk', 0)

In [214]:
i = movie_reviews.vocab.stoi['suvarnabhumi']
movie_reviews.vocab.itos[i], i

('xxunk', 0)

In [215]:
i = movie_reviews.vocab.stoi['airport']
movie_reviews.vocab.itos[i], i

('airport', 4978)

เลือก x ของ Record แรก ของ Training Set 

In [224]:
t = movie_reviews.train[0][0]
t

Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !

Token ภายใน x จะถูก Numberize แปลงเป็นตัวเลขเรียบร้อยแล้ว

In [217]:
t.data[:30]

array([   2,    5, 4619,   25,    0,   25,  867,   52,    5, 3776,    5, 1800,   95,   37,   85,  191,   64,  935,
          0, 2738,  517,   18,   21,   11,   84, 2417,  192,   88, 3777,   64])

# 4. Creating Term-Document Matrix

## 4.1 Counter

In [225]:
c = Counter([1, 2, 3, 4, 5, 
             1, 2, 3, 4, 
             1, 2, 3, 
             1, 2, 
             1])
c

Counter({1: 5, 2: 4, 3: 3, 4: 2, 5: 1})

In [226]:
c.keys(), c.values()

(dict_keys([1, 2, 3, 4, 5]), dict_values([5, 4, 3, 2, 1]))

## 4.2 Term-Document Matrix as Sparse Matrix

What is Sparse Matrix?

In [227]:
Counter(movie_reviews.valid.x[0].data)

Counter({0: 32,
         2: 1,
         5: 32,
         6: 1,
         9: 10,
         10: 7,
         11: 10,
         12: 1,
         13: 4,
         14: 6,
         15: 6,
         16: 4,
         18: 2,
         20: 1,
         21: 3,
         23: 1,
         24: 3,
         25: 2,
         26: 1,
         27: 3,
         30: 1,
         44: 1,
         45: 1,
         49: 1,
         50: 3,
         52: 1,
         54: 2,
         58: 1,
         59: 1,
         63: 2,
         71: 1,
         74: 1,
         77: 1,
         84: 1,
         109: 1,
         115: 1,
         149: 1,
         189: 1,
         194: 1,
         197: 2,
         204: 1,
         207: 1,
         221: 1,
         239: 1,
         251: 1,
         258: 1,
         285: 1,
         288: 1,
         319: 1,
         324: 1,
         337: 1,
         358: 1,
         378: 1,
         404: 1,
         409: 1,
         430: 1,
         456: 1,
         478: 1,
         541: 1,
         571: 1,
         579: 1

In [228]:
idx = [0, 2, 5, 6, 9]
[movie_reviews.vocab.itos[i] for i in idx]

['xxunk', 'xxbos', 'xxmaj', 'xxup', 'the']

In [229]:
movie_reviews.valid.y[1], movie_reviews.valid.x[1]

(Category positive,
 Text xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
  
   xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
  
   xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk)

In [230]:
movie_reviews.valid.y[1].data, movie_reviews.valid.x[1].data

(1, array([  2,  19, 248,  21, ...,   9,   0,  10,   0]))

In [0]:
def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)

    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))

    # return (values, j_indices, indptr)

    return scipy.sparse.csr_matrix((values, j_indices, indptr), 
                                   shape=(len(indptr)-1, vocab_len), 
                                   dtype=int)

In [239]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

CPU times: user 47.1 ms, sys: 2.2 ms, total: 49.3 ms
Wall time: 45.9 ms


In [240]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

CPU times: user 175 ms, sys: 12.6 ms, total: 188 ms
Wall time: 173 ms


In [241]:
trn_term_doc.shape

(800, 6008)

In [242]:
val_term_doc.shape

(200, 6008)

## 4.3 Sparse Matrix vs Dense Matrix

In [243]:
trn_term_doc[:, -10:]

<800x10 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [248]:
trn_term_doc.todense()[:8, :8]

matrix([[ 8,  0,  1,  0,  0, 15,  0,  0],
        [22,  0,  1,  0,  0, 33,  2,  0],
        [ 4,  0,  1,  0,  0, 16,  2,  0],
        [13,  0,  1,  0,  0, 11,  0,  0],
        [12,  0,  1,  0,  0, 21,  0,  0],
        [ 4,  0,  1,  0,  0, 12,  1,  0],
        [ 4,  0,  1,  0,  0, 15,  0,  0],
        [42,  0,  1,  0,  0, 43, 14,  0]])

## 4.4 Data Exploration

In [0]:
v = movie_reviews.vocab

In [285]:
v.itos[0]

'xxunk'

In [286]:
v.itos[-1:]

['sollett']

In [287]:
v.itos[3]

'xxeos'

เราจะมาดูตัวอย่างข้อความ Record ที่ 1 ใน Validation Set มีคำว่า Late 2 คำ ในข้อความ

In [251]:
review = movie_reviews.valid.x[1]; review

Text xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
 
  xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
 
  xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk

คำว่า Late คือคำที่ 451 ใน vocab Dictionary

In [252]:
i = v.stoi['late']; i

451

ดูใน Validation Term-Document Matrix ที่ Row 1, Column 451 จะได้ 2

In [253]:
val_term_doc[1, 451]

2

มีทั้งหมด 144 Token จาก 81 คำศัพท์

In [254]:
val_term_doc[1].sum(), (val_term_doc[1] > 0).sum()

(144, 81)

Token ที่ Numberize แล้ว

In [255]:
review.data

array([  2,  19, 248,  21, ...,   9,   0,  10,   0])

แปลง Token ที่ Numberize แล้ว กลับเป็น Token ข้อความ

In [256]:
[v.itos[i] for i in review.data][:20]

['xxbos',
 'i',
 'saw',
 'this',
 'movie',
 'once',
 'as',
 'a',
 'kid',
 'on',
 'the',
 'late',
 '-',
 'late',
 'show',
 'and',
 'fell',
 'in',
 'love',
 'with']

## 4.5 Unknown Words

จำนวน itos ไม่เท่ากับ stoi เนื่องจาก หลายค่ำกลายเป็น Unknown เพื่อลดขนาด vocab Dictionary

In [257]:
len(v.itos), len(v.stoi)

(6008, 19163)

ต่างกันถึง 13155 คำ

In [258]:
len(v.stoi) - len(v.itos)

13155

ดูรายการ Unknown Word ทั้งหมด

In [260]:
unk = [k for k, v in v.stoi.items() if v == 0]
len(unk), unk[:20]

(13156,
 ['xxunk',
  'bleeping',
  'pert',
  'ticky',
  'schtick',
  'whoosh',
  'banzai',
  'chill',
  'wooofff',
  'cheery',
  'superstars',
  'fashionable',
  'cruelly',
  'separating',
  'mistreat',
  'tensions',
  'religions',
  'baseness',
  'nobility',
  'puro'])

# 5. Sentiment Classification

## 5.1 Naive Bayes


We define the log-count ratio $r$ for each word $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive document has a feature divided by the number of negative documents.

In [277]:
movie_reviews.y.classes

['negative', 'positive']

In [0]:
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [279]:
positive = y.c2i['positive']
negative = y.c2i['negative']

positive, negative

(1, 0)

In [280]:
np.squeeze(np.asarray(x[y.items==negative].sum(0)))

array([7154,    0,  417,    0, ...,    0,    3,    3,    3], dtype=int64)

In [281]:
np.squeeze(np.asarray(x[y.items==positive].sum(0)))

array([6471,    0,  383,    0, ...,    3,    0,    0,    0], dtype=int64)

### For each words sum positive and negative reviews.

In [282]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

len(p1), len(p0)

(6008, 6008)

In [288]:
p1[:10], p0[:10]

(array([ 6471,     0,   383,     0,     0, 10267,   674,    57,     0,  5260], dtype=int64),
 array([ 7154,     0,   417,     0,     0, 10741,   908,    53,     1,  6150], dtype=int64))

### Explore Ratio

In [0]:
def p_ratio(s):
    i = v.stoi[s]
    return p1[i]/p0[i]

In [292]:
s = 'loved'
p_ratio(s)

2.4166666666666665

In [293]:
s = 'hated'
p_ratio(s)

0.5

In [295]:
s = 'best'
p_ratio(s)

1.369047619047619

In [296]:
s = 'worst'
p_ratio(s)

0.05333333333333334

### Explore data

Positive review with hated.

In [297]:
v.stoi['hated']

1977

In [299]:
a = np.argwhere((x[:, 1977] > 0))[:, 0]; a

array([ 15,  49, 304, 351, 393, 612, 695, 773], dtype=int32)

In [300]:
b = np.argwhere(y.items==positive)[:, 0]; b

array([  1,   3,  10,  11, ..., 787, 789, 790, 797])

In [302]:
set(a).intersection(set(b))

{393, 612, 695}

In [311]:
review = movie_reviews.train.x[393]
review.text

'xxbos xxmaj there are numerous films relating to xxup xxunk , but xxmaj mother xxmaj night is quite xxunk among them : xxmaj in this film , we are introduced to xxmaj howard xxmaj campbell ( xxmaj nolte ) , an xxmaj american living in xxmaj berlin and married to a xxmaj german , xxmaj xxunk xxmaj xxunk ( xxmaj lee ) , who decides to accept the role of a spy : xxmaj more specifically , a xxup cia agent xxmaj major xxmaj xxunk ( xxmaj goodman ) recruits xxmaj campbell who becomes a xxmaj nazi xxunk in order to enter the highest xxunk of the xxmaj hitler xxunk . xxmaj however , the deal is that the xxup us xxmaj government will never xxunk xxmaj campbell \'s role in the war for national security reasons , and so xxmaj campbell becomes a hated figure across the xxup us . xxmaj after the war , he tries to xxunk his identity , but the past comes back and xxunk him . xxmaj his only " friend " is xxmaj xxunk , but even he can not do much for the xxunk of events that fall upon poor xxmaj campb

In [312]:
review = movie_reviews.train.x[612]
review.text

'xxbos xxmaj once upon a time in a castle xxrep 6 . xxmaj two little girls are playing in the garden \'s castle . xxmaj they are sisters . a blonde little girl ( xxmaj kitty ) and a brunette one ( xxmaj evelyn ) . xxmaj evelyn steals xxmaj kitty \'s doll . xxmaj kitty xxunk xxmaj evelyn . xxmaj running through long xxunk , they reach the room where their grandfather , sitting on an xxunk , reads the newspaper . xxmaj kitty xxunk about xxmaj evelyn , while xxmaj evelyn is looking xxunk at a picture hanging on the wall . xxmaj evelyn begins to say repeatedly : " i am the red lady and xxmaj kitty is the black lady " . xxmaj suddenly xxmaj evelyn grabs a dagger lying nearby and stabs xxmaj kitty \'s doll and then cuts her ( the doll \'s ) head . a fight ensues . xxmaj and xxmaj evelyn almost uses the dagger against xxmaj kitty . xxmaj the grandfather xxunk and the worst is avoided . \n \n  xxmaj later on , their grandfather tells them the legend related to the picture hanging on the wall i

In [313]:
review = movie_reviews.train.x[695]
review.text

"xxbos xxmaj xxunk , yeah this episode is extremely underrated . \n \n  xxmaj even though there is a xxup lot of bad writing and acting at parts . i think the good over wins the bad . \n \n  i love the xxunk parts and the big ' twist ' at the end . i absolutely love that scene when xxmaj michelle xxunk xxmaj tony . xxmaj it 's actually one of my favorite scenes of xxmaj season 1 . \n \n  xxmaj for some reason , people have always hated the xxmaj xxunk episodes , yet i have always liked them . xxmaj they 're not the best , in terms of writing . but the theme really does interest me , \n \n  i 'm gon na give it a xxup three star , but if the writing were a little more consistent i 'd give it xxup four ."

Negative review with loved

In [314]:
v.stoi['loved']

535

In [315]:
a = np.argwhere((x[:, 535] > 0))[:, 0]; a

array([  1,  15,  29,  69,  75,  79, 174, 185, 200, 205, 262, 296, 303, 333, 350, 351, 398, 407, 440, 489, 496, 528,
       538, 600, 602, 605, 627, 642, 657, 660, 700, 712, 729, 735, 755, 767, 785], dtype=int32)

In [316]:
b = np.argwhere(y.items==negative)[:, 0]; b

array([  0,   2,   4,   5, ..., 795, 796, 798, 799])

In [317]:
set(a).intersection(set(b))

{15, 200, 205, 303, 351, 398, 600, 605, 642, 700, 729, 767}

In [318]:
review = movie_reviews.train.x[15]
review.text

"xxbos xxmaj as if the storyline was n't depressing enough , this movie shows xxunk being xxunk graphically in a slaughterhouse for all of five minutes while the protagonist is xxunk her early life as a butcher . xxmaj weird stuff . xxmaj then there 's the core premise of the hero / heroine who goes and cuts his dick off because a he 's xxunk - ten with at work says he would have gone with him if he was a girl . xxmaj is this person a psycho , a xxunk , just a doomed queen who takes things too far ? xxmaj and what sort of xxunk childhood did he have ? xxmaj just that he did n't get adopted and had to live it out with xxunk who at first loved him and then later hated him because he was xxunk . xxmaj he tries to explain to us the reasons he did what he did , but it 's really really so hard to xxunk . xxmaj such sad and unusual self destruction . xxmaj was it supposed to be funny ? xxmaj what was it all about really ?"

In [319]:
review = movie_reviews.train.x[200]
review.text

'xxbos i loved the first " xxmaj azumi " movie . i \'ve seen xxmaj ms. xxmaj ueto in a variety of her xxup tv appearances and i \'ve seen my fair share of samurai and ninja flicks . i have to say that this movie was much weaker than i \'d expected . \n \n  xxmaj given the movie \'s cast and set up in " xxmaj azumi " , they should have been able to do a much better job with this movie , but instead it was slow , xxunk in parts , and xxunk with very poor , unconvincing , and wooden acting . \n \n  xxmaj when they bothered to reference the first movie , they did so in a manner that was pretty loose and weak . xxmaj in " xxmaj azumi " , the title character is the best of a group of superior killers . xxmaj in " xxmaj azumi 2 " she seems somehow xxunk and less - impressive . \n \n  xxmaj that \'s not to say it was a total loss . xxmaj there were a few decent fight scenes and some over - the - top characters . xxmaj unfortunately , the movie suffers overall from the simple fact that xxmaj xx

In [320]:
review = movie_reviews.train.x[205]
review.text

'xxbos xxunk is normally xxunk , and it \'s never really bothered me before -- i loved " xxunk 2 " , but i really do n\'t like pseudo xxunk or playing on xxunk . the whole movie is based on a man with esp caused by an xxunk accident and a witch . i \'m not opposed to xxunk , and i liked " carrie " ( the novel and the movie ) but this one bothered me . i think it \'s because of the main character developing esp from an electric line . also the university professor wasting his time studying esp cases . i was n\'t alive in the 60s to know first hand whether or not esp was a common xxunk then , but i assume that any theory of such nature would simply be xxunk . what really bothered me was the way the police were xxunk xxunk as xxunk , xxunk villains who can seem like normal people 99 % of the time and then just snap . nothing could be further from the truth . i xxunk such xxunk because they add to public xxunk . many people still think that xxunk is xxunk identity xxunk . whoever wrote thi

### เข้าสูตร Naive Bayes

In [321]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

len(p1), len(p0)

(6008, 6008)

In [322]:
pr1 = (p1 + 1) / ((y.items==positive).sum() + 1)
pr0 = (p0 + 1) / ((y.items==negative).sum() + 1)

len(pr1), len(pr0)

(6008, 6008)

In [324]:
r = np.log(pr1/pr0); r

array([-0.015487,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

ดูคำศัพท์ที่มักใช้ใน Positive / Negative Review

In [0]:
biggest = np.argpartition(r, -10)[-10:]
smallest = np.argpartition(r, 10)[:10]

คำที่ Positive ที่สุด

In [331]:
[v.itos[i] for i in biggest]

['sport',
 'davies',
 'gilliam',
 'fanfan',
 'biko',
 'felix',
 'noir',
 'jabba',
 'astaire',
 'jimmy']

In [329]:
np.argmax(trn_term_doc[:, v.stoi['biko']])

515

In [330]:
movie_reviews.train.x[515]

Text xxbos " xxmaj the xxmaj true xxmaj story xxmaj of xxmaj the xxmaj friendship xxmaj that xxmaj shook xxmaj south xxmaj africa xxmaj and xxmaj xxunk xxmaj the xxmaj world . " 
 
  xxmaj richard xxmaj attenborough , who directed " a xxmaj bridge xxmaj too xxmaj far " and " xxmaj gandhi " , wanted to bring the story of xxmaj steve xxmaj biko to life , and the journey and trouble that xxunk xxmaj donald xxmaj woods went through in order to get his story told . xxmaj the films uses xxmaj wood 's two books for it 's information and basis - " xxmaj biko " and " xxmaj asking for xxmaj trouble " . 
 
  xxmaj the film takes place in the late 1970 's , in xxmaj south xxmaj africa . xxmaj south xxmaj africa is in the grip of the terrible apartheid , which keeps the blacks separated from the whites and xxunk the whites as the superior race . xxmaj the blacks are forced to live in xxunk on the xxunk of the cities and xxunk , and they come under frequent xxunk by the police and the army . xxmaj w

คำที่ Negative ที่สุด

In [327]:
[v.itos[i] for i in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

In [332]:
np.argmax(trn_term_doc[:, v.stoi['soderbergh']])

434

In [333]:
movie_reviews.train.x[434]

Text xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of " xxmaj at xxmaj the xxmaj movies " in taking xxmaj steven xxmaj soderbergh to task . 
 
  xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after xxunk years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside " edgy " projects . 
 
  xxmaj none of this excuses him this present , almost diabolical failure . xxmaj as xxmaj david xxmaj xxunk xxunk , " two parts of xxmaj che do n't ( even ) make a whole " . 
 
  xxmaj epic xxunk in name only , xxmaj che(2008 ) barely qualifies as a feature film 

Predict with Bayes

In [334]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [336]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()) ; b

-0.08505123261815539

In [0]:
preds = (val_term_doc @ r + b ) > 0

In [338]:
(preds == val_y.items).mean()

0.645